In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import scipy.stats as s 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data=pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')

In [ ]:
data

In [ ]:
data.shape[0]

In [ ]:
training_data_len = int(0.7*data.shape[0])

In [ ]:
training_data_len

In [ ]:
cv_data_len = int(0.2*data.shape[0])


In [ ]:
cv_data_len

In [ ]:
testing_data_len = int(0.1*data.shape[0])


In [ ]:
testing_data_len


In [ ]:
poissonous_train = data[data['class'] == 'p'].iloc[0:training_data_len//2]
non_poissonous_train = data[data['class'] == 'e'].iloc[0:training_data_len//2]

In [ ]:
training_data = pd.concat([poissonous_train,non_poissonous_train])

In [ ]:
training_data.shape

In [ ]:
poissonous_remain = data[data['class'] == 'p'].iloc[training_data_len//2:]
non_poissonous_remain = data[data['class'] == 'e'].iloc[training_data_len//2:]

In [ ]:
remaining_data = pd.concat([poissonous_remain,non_poissonous_remain])

In [ ]:
remaining_data.shape

In [ ]:
remaining_data

In [ ]:
remaining_data = np.array(remaining_data)

In [ ]:
remaining_data

In [ ]:
random_indices = np.random.choice(np.arange(0,2438),size=(2438,))

In [ ]:
remaining_data = remaining_data[random_indices]

In [ ]:
remaining_data

In [ ]:
remaining_data = pd.DataFrame(data=remaining_data,columns=data.columns)

In [ ]:
remaining_data

In [ ]:
cv_data = remaining_data.iloc[0:cv_data_len]
testing_data = remaining_data.iloc[cv_data_len:]

In [ ]:
cv_data.shape

In [ ]:
testing_data.shape

In [ ]:
cv_data

In [ ]:
testing_data

In [ ]:
def lidstone_smoothing(column_name,category,alpha):
    
    D = dict()
    
    train_unique_feature_values = training_data[training_data['class'] == category][column_name].unique()
    
    data_unique_feature_values = data[column_name].unique()
    
    for feature_value in data_unique_feature_values:
        
        if feature_value not in train_unique_feature_values:
            
            D[feature_value] = alpha/(training_data[training_data['class'] == category].shape[0] + (data_unique_feature_values.shape[0]*alpha))
            
        else:
            
            D[feature_value] = np.count_nonzero(training_data[training_data['class'] == category][column_name] == feature_value)/training_data[training_data['class'] == category].shape[0]
            
    return D

In [ ]:
D_of_D = {'p':{},'e':{}}

for category in ['p','e']:
    
    for column_name in data.columns[1:]:
        
        D_of_D[category][column_name] = lidstone_smoothing(column_name,category,0.8)

In [ ]:
D_of_D

In [ ]:
prior_p = training_data[training_data['class'] == 'p'].shape[0]/training_data.shape[0]
prior_e = training_data[training_data['class'] == 'e'].shape[0]/training_data.shape[0]

In [ ]:
def mock_test(data):
    
    likelihood_p = 1
    
    likelihood_e = 1
    
    for k,v in data.items():
                                
        likelihood_p = likelihood_p * D_of_D['p'][k][v]
    
        likelihood_e = likelihood_e * D_of_D['e'][k][v]
        
    posterior_p = likelihood_p * prior_p
    
    posterior_e = likelihood_e * prior_e
    
    if posterior_p > posterior_e:
        
        return 'p'
    
    else:
        
        return 'e'

In [ ]:
actual_results = np.array(cv_data['class'])

cv_data = np.array(cv_data.iloc[:,1:])

keys = data.columns[1:]

predicted_category = list()

for i in range(0,cv_data.shape[0]):
    
    data = dict(zip(keys,list(cv_data[i])))
    
    predicted_category.append(mock_test(data))

In [ ]:
predicted_category = np.array(predicted_category)

In [ ]:
predicted_category

In [ ]:
actual_results

In [ ]:
accuracy = np.count_nonzero(predicted_category == actual_results)/actual_results.shape[0]

In [ ]:
accuracy